In [ ]:
%%capture
# !pip install unsloth "xformers==0.0.28.post2"
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install langchain
!pip install transformers
!pip install accelerate
!pip install sentencepiece
!pip install langchain_community
!pip install xformers

In [ ]:
import torch
from unsloth import FastLanguageModel
from langchain.llms.base import LLM
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from typing import Optional, List
from pydantic import Field

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    device_map="auto"
)
FastLanguageModel.for_inference(model)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
class UnslothLLM(LLM):
    model: any = Field(...)
    tokenizer: any = Field(...)

    @property
    def _llm_type(self) -> str:
        return "unsloth"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        inputs = self.tokenizer(prompt, return_tensors="pt").to(device)
        output = self.model.generate(**inputs, max_new_tokens=50)
        response = self.tokenizer.decode(output[0], skip_special_tokens=True)
        return response

In [ ]:
prompt = PromptTemplate(
    input_variables=["input_text"],
    template="Given the input: '{input_text}', provide a detailed response."
)

unsloth_llm = UnslothLLM(model=model, tokenizer=tokenizer)

chain = LLMChain(
    llm=unsloth_llm,
    prompt=prompt
)

In [ ]:
input_text = "Explain how Unsloth can be integrated with LangChain in Python."

response = chain.run(input_text=input_text)

print(response)